**Boston House Predict **
I love to share with you guys, I still novie, so I am glad to hear your advice

S**o, this is the description about data :**

crim : per capita crime rate by town.

zn : proportion of residential land zoned for lots over 25,000 sq.ft.

indus : proportion of non-retail business acres per town.

chas : Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

nox : nitrogen oxides concentration (parts per 10 million).

rm : average number of rooms per dwelling.

age : proportion of owner-occupied units built prior to 1940.

dis : weighted mean of distances to five Boston employment centres.

rad : index of accessibility to radial highways.

tax : full-value property-tax rate per \$10,000.

ptratio : pupil-teacher ratio by town.

black : 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

lstat :lower status of the population (percent).

medv : median value of owner-occupied homes in \$1000s.

Target Variable = medv
---------------------------------------------

Before start, I want you to know, I write this in two bahasa, Inggris and Indonesia

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

import sklearn as sk
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


In [ ]:
data_latih = pd.read_csv('../input/boston_train.csv', sep=',', index_col = ['ID'])
target = data_latih['medv']
print("Search For Missing Values")
print(data_latih.isnull().sum())
data_latih = data_latih.drop(['medv'], axis=1)
original_data = data_latih

Lima baris pertama

In [ ]:
print(data_latih.head())

In [ ]:
print(data_latih.describe())

In [ ]:
print(data_latih.shape)

ampai sini kita dapat melihat bahwa hampir semua data berjenis numeric, namun sebenarnya ada satu jenis berjenis kategori, yaitu data pada feature chas. yang dinyatakan dalam dummy variable. sepertinya kita perlu mengubah feature ini menjadi dummy variable secara terpisah (tidak dalam satu kolom). dibandingkan harus disatukan menjadi data order. karena itu akan memberikan perbedaan pada output. since we don't know one is better than other.

In [ ]:
plt.figure(figsize=(7,7))
sns.heatmap(data_latih.corr(), square=True)
plt.show()

In heatmap table, we see tax and rad has strong correlation, let's explore about these two variables

In [ ]:
print(original_data[['tax', 'rad']].head(8))

indeks aksesibilitas ke jalan raya semakin tinggi maka harga pajakpun semakin besar

In [ ]:
rata2=target.mean()
col = []
for n in target[:] :
    if n>=rata2 :
        col.append('green')
    else :
        col.append('red')

col = pd.Series(col)
pd.plotting.scatter_matrix(original_data, c = col, figsize=(30,30), marker='.')
plt.show()

Kita lihat dari gambar visualisasi diatas, warna hijau menunjukan harga yang diatas rata-rata dan harga merah menunjukan harga yang dibawah rata-rata...
disana kita dapat mudah melihat variasi nilai variable terhadap harga rumah...
kita dapat menarik banyak kesimpulan dari gambar-gambar diatas, diantarnya :
- tingkat kriminalitas tidak bisa ditolelir, tidak ada satu pun rumah yang berada di kota yang mempunyai tingkat kriminalitas besar, memiliki harga diatas rata-rata, tidak peduli bagaimana variable lain. ini menunjukan bahwa kriminalitas mempunyai korelasi kuat negatif terhadap harga rumah. walaupun variansi dari variable ini kecil, tapi hampir semua rumah yang mempunyai harga di bawah rata2 memiliki tingkat kriminalitas yang lebih besar di bandingkan rumah yang harganya di atas rata2
- zn = tingkat keramaian penduduk, mostly, zn has price above the average if the zn is high. zn mempunyai korelasi phositif.
- rumah yang berada di daerah banyak pabrik, mempunyai harga yang murah dibandingkan dengan rumah yang sedikit harga pabrik, namun dalam kasus khusus, adapula rumah yang murah walaupun tidak terdapat banyak pabrik in that town, that's mean rumah mempunyai korelasi phositif
- ..........

dan seterusnya, dengan scatter_matrix ini kita dapat meraba pengaruh setiap variable terhadap harga rumah.
salah satu variable yang tampaknya memiliki pengaruh lemah terhadap harga rumah, adalah variable black

Now, Let's we do machine Learning


In [ ]:
## Linier Regression
linreg = LinearRegression(normalize=True)
linreg.fit(original_data, target)
r2 = linreg.score(original_data, target)

#importing Test Data
data_test = pd.read_csv('../input/boston_train.csv', sep=',', index_col = ['ID'])
test_target = data_test['medv']
data_test=data_test.drop(['medv'], axis=1)

y_pred=linreg.predict(data_test)
mse = mean_squared_error(test_target, y_pred)
rmse=np.sqrt(mse)
print(r2)
print(rmse)

In [ ]:
#Lasso Regression
alphas=np.linspace(0.001, 1, 100)
lass = Lasso(normalize=True)
lasso_scores = []
lasso_scores_std = []
rmse=[]
for alpha in alphas:

    # Specify the alpha value to use: ridge.alpha
    lass.alpha = alpha
    
    # Perform 10-fold CV: ridge_cv_scores
    lasso_cv_scores = cross_val_score(lass, original_data, target, cv=5)
    
    # Append the mean of ridge_cv_scores to ridge_scores
    lasso_scores.append(np.mean(lasso_cv_scores))
    
    # Append the std of ridge_cv_scores to ridge_scores_std
    lasso_scores_std.append(np.std(lasso_cv_scores))
    

#Display Plot Function
def display_plot(cv_scores, cv_scores_std):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(alphas, cv_scores)

    std_error = cv_scores_std / np.sqrt(10)

    ax.fill_between(alphas, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
    ax.set_ylabel('CV Score +/- Std Error')
    ax.set_xlabel('Alpha')
    ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
    ax.set_xlim([alphas[0], alphas[-1]])
    ax.set_xscale('log')
    plt.show()
    
# Display the plot
display_plot(lasso_scores, lasso_scores_std)



1. Any commend with that result above?

In [ ]:
model = Lasso(alpha=0.001, normalize=True)
model.fit(original_data, target)
r2=model.score(original_data, target)
test_pred=model.predict(data_test)
rmse=np.sqrt(mean_squared_error(test_target, test_pred))
coefs = model.coef_
intercept = model.intercept_
print("Intercept : {}".format(intercept))
print("RMSE : {}".format(rmse))
print("R2 : {}".format(r2))

Not bad, our RMSE is 4.7 with accuracy R2

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(range(len(original_data.columns)), coefs)
plt.xticks(range(len(original_data.columns)), original_data.columns, rotation=60)
plt.margins(0.02)
plt.show()

So, jumlah ruangan dan chas mempunyai korelasi positif paling besar dengan harga rumah
dan jumlah nitrogen mempunyai korelasi negatif paling besar dengan harga rumah